<a href="https://colab.research.google.com/github/MoRushaid/HealthCostCalc/blob/main/predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from sklearn import tree
from sklearn.model_selection import train_test_split
import seaborn as sns


In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
device_name = tf.test.gpu_device_name()
if len(device_name) > 0:
    print("Found GPU at: {}".format(device_name))
else:
    device_name = "/device:CPU:0"
    print("No GPU, using {}.".format(device_name))

In [ ]:
dataset["sex"] = pd.factorize(dataset["sex"])[0]
dataset["region"] = pd.factorize(dataset["region"])[0]
dataset["smoker"] = pd.factorize(dataset["smoker"])[0]

In [ ]:
#dataset.describe()
features = ['age','bmi','children','sex','smoker','region']
X = dataset.loc[:, features]
y = dataset.loc[:, ['expenses']]
train_dataset, test_dataset, train_labels, test_labels = train_test_split(X, y, random_state=0, train_size = .8)

In [ ]:
train_labels.head()

In [ ]:
#features_cat = ['sex', 'smoker', 'region']
#for feature in features_cat:
#    train_dataset[pd.get_dummies(train_dataset[feature], prefix=feature).columns] = pd.get_dummies(train_dataset[feature], prefix=feature)
#    test_dataset[pd.get_dummies(test_dataset[feature], prefix=feature).columns] = pd.get_dummies(test_dataset[feature], prefix=feature)
#train_dataset.drop(['sex','smoker','region'], axis = 1, inplace = True)
#test_dataset.drop(['sex','smoker','region'], axis = 1, inplace = True)

In [ ]:
train_dataset.head()

In [ ]:
with tf.device(device_name):

  normalizer = layers.experimental.preprocessing.Normalization()
  normalizer.adapt(np.array(train_dataset))

  model = keras.Sequential([
          normalizer,
          layers.Dense(64, activation='relu'),
          layers.Dropout(.2),
          layers.Dense(64, activation='relu'),
          layers.Dropout(.2),
          layers.Dense(1),
          ])

  model.compile(
          optimizer = tf.optimizers.Adam(learning_rate=0.001),
          loss = 'mae',
          metrics = ['mae', 'mse']
          )
  model.build()

model.summary()

In [ ]:
history = model.fit(
    train_dataset,
    train_labels,
    epochs = 100,
    validation_split = 0.15,
    verbose=0
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
